In [ ]:
# def write_to_record_matrix(self, thread_idx, record_idx, data):
#     tx_pos = self.position_controller.controller.position["xy"][
#         self.yaml_config["emitter"]["motor_channel"]
#     ]
#     rx_pos = self.position_controller.controller.position["xy"][
#         self.rx_configs[0].motor_channel
#     ]

#     data.tx_pos_x_mm = tx_pos[0]
#     data.tx_pos_y_mm = tx_pos[1]
#     data.rx_pos_x_mm = rx_pos[0]
#     data.rx_pos_y_mm = rx_pos[1]
#     assert data.rx_lo > 1

#     if not self.yaml_config["dry-run"]:
#         z = self.zarr[f"receivers/r{thread_idx}"]
#         z.signal_matrix[record_idx] = data.signal_matrix
#         for k in v5rx_f64_keys + v5rx_2xf64_keys:
#             z[k][record_idx] = getattr(data, k)

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset

from spf.dataset.v5_data import v5rx_2xf64_keys, v5rx_f64_keys, v5rx_new_dataset

ds = v5spfdataset(
    "/mnt/md2/2d_wallarray_v2_data/april_nuand/wallarrayv3_2025_04_04_21_06_04_nRX2_rx_random_circle_spacing0p08.zarr",
    nthetas=65,
    ignore_qc=True,
    precompute_cache="/mnt/md2/cache/precompute_cache_3p5_chunk1/",
    gpu=False,
    snapshots_per_session=1,
    n_parallel=8,
    paired=True,
    segmentation_version=3.5,
)

In [ ]:
# for idx in range(100):
#     ds[idx]

In [ ]:
v5_raw_keys = v5rx_f64_keys + v5rx_2xf64_keys + ["signal_matrix"]


def data_single_radio_to_raw(d):
    return {k: d[k] for k in v5_raw_keys}

In [ ]:
ds[0][0]["mean_phase_segmentation"].shape

In [ ]:
from spf.dataset.segmentation import DEFAULT_SEGMENT_ARGS, segment_session

v5inf = v5inferencedataset(
    yaml_fn="/mnt/md2/2d_wallarray_v2_data/april_nuand/wallarrayv3_2025_04_04_21_06_04_nRX2_rx_random_circle_spacing0p08.yaml",
    nthetas=65,
    gpu=False,
    n_parallel=8,
    paired=True,
    model_config_fn="",
    vehicle_type="wallarray",
    skip_segmentation=True,
)
a = ds[0][0]

In [ ]:
a["receiver_idx"]

In [ ]:
b = v5inf.write_to_idx(0, 0, data_single_radio_to_raw(a))
a = 1

In [ ]:
a["simple_segmentations"][0]

In [ ]:
def compare_two_entries(a, b):
    for k, v in a.items():
        if k in training_only_keys:
            continue
        assert k in b
        vp = b[k]
        print(k)
        if isinstance(v, torch.Tensor):
            assert v.isclose(vp, rtol=1e-3).all(), f"{v} {vp}"
        elif isinstance(v, List):
            s = v[0]
            sp = vp[0]
            assert len(s) == len(sp)
            for idx in range(len(s)):
                assert s[idx]["start_idx"] == sp[idx]["start_idx"]
                assert s[idx]["end_idx"] == sp[idx]["end_idx"]

In [ ]:
ds_fn = "/mnt/md0/2d_wallarray_v2_data/oct_batch2/wallarrayv3_2024_10_26_01_14_28_nRX2_rx_circle_spacing0p075.zarr"

ds1 = v5spfdataset(
    ds_fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache="/mnt/md2/cache/precompute_cache_3p5_chunk1/",
    gpu=False,
    snapshots_per_session=1,
    n_parallel=8,
    paired=True,
    segmentation_version=3.5,
)

In [ ]:
ds2 = v5spfdataset(
    ds_fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache="/mnt/md2/cache/precompute_cache_3p6/",
    gpu=False,
    snapshots_per_session=1,
    n_parallel=8,
    paired=True,
    segmentation_version=3.6,
)

In [ ]:
ds3 = v5spfdataset(
    ds_fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache="/mnt/md2/cache/precompute_cache_3p6x/",
    gpu=False,
    snapshots_per_session=1,
    n_parallel=8,
    paired=True,
    segmentation_version=3.6,
)

In [ ]:
def comparez(a, b):
    for k, v in a.items():
        if k not in b:
            print("B missing", k)
        else:
            vb = b[k]
            if isinstance(v, torch.Tensor):
                if (~v.isfinite()).all():
                    pass  # all infinite
                elif (v.isclose(vb, rtol=1e-3) * 1.0).mean() > 0.99:
                    # print(k,"pass")
                    # print((v.isclose(vb,rtol=1e-3)*1.0).mean())
                    pass
                else:
                    # print((v.isclose(vb,rtol=1e-3)*1.0).mean())
                    print(k, "fail", (v.isclose(vb) * 1.0).mean(), v, vb)


for idx in range(30):
    for ridx in range(2):
        z1 = ds1[idx][ridx]
        z2 = ds2[idx][ridx]
        z3 = ds3[idx][ridx]
        comparez(z1, z2)
        comparez(z2, z3)

In [ ]:
~torch.tensor(torch.inf).isfinite()

In [ ]:
ds2[0][0]["windowed_beamformer"]

In [ ]:
ds1[0][0]["windowed_beamformer"]

In [ ]:
from spf.dataset.spf_dataset import training_only_keys
import torch

from typing import List


def compare_two_entries(a, b):
    for k, v in a.items():
        if k in training_only_keys + ["craft_y_rad_binned"]:
            continue
        assert k in b, f"{k} missing!"
        vp = b[k]
        print(k)
        if isinstance(v, torch.Tensor):
            assert v.isclose(vp, rtol=1e-2).all(), f"{v} {vp}"
        elif isinstance(v, List):
            s = v[0]
            sp = vp[0]
            assert len(s) == len(sp)
            for idx in range(len(s)):
                assert s[idx]["start_idx"] == sp[idx]["start_idx"]
                assert s[idx]["end_idx"] == sp[idx]["end_idx"]

In [ ]:
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

import pickle
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset


n = 50
tmpdirname = "/tmp/test1"
fn = tmpdirname + f"/perfect_circle_n{n}_noise0p0"

create_fake_dataset(filename=fn, yaml_config_str=fake_yaml_v4, n=50, noise=0.01)

ds = v5spfdataset(  # make sure everything gets segmented here
    fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    # skip_fields=set(["signal_matrix"]),
    segment_if_not_exist=True,
    v4=True,
)

v5inf = v5inferencedataset(
    yaml_fn=fn + ".yaml",
    nthetas=65,
    gpu=False,
    paired=True,
    model_config_fn="",
    skip_fields=[],
    vehicle_type="wallarray",
    skip_segmentation=False,
    skip_detrend=False,
)
n = 3

for idx in range(n):
    d = ds[idx]
    for ridx in range(2):
        print(idx, ridx)
        v5inf.write_to_idx(idx, ridx, data_single_radio_to_raw(d[ridx]))
        compare_two_entries(ds[0][1], v5inf[0][1])

In [ ]:
ds[0][0].keys()

In [ ]:
ds[0][0]["craft_y_rad"], v5inf[0][0]["craft_y_rad"]

In [ ]:
from spf.utils import SEGMENTATION_VERSION, load_config, rx_spacing_to_str, to_bin

to_bin(v5inf[0][0]["craft_y_rad"], 65)

In [ ]:
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

# create_fake_dataset(filename=fn, yaml_config_str=fake_yaml_v4, n=50, noise=0.01)

n = 50
tmpdirname = "/tmp/test1"
fn = tmpdirname + f"/perfect_circle_n{n}_noise0p0"
ds = v5spfdataset(  # make sure everything gets segmented here
    fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    # skip_fields=set(["signal_matrix"]),
    segment_if_not_exist=True,
    v4=True,
)

n = 20
precomputed_ds = [ds[i] for i in range(20)]
import pickle

pickle.dump(precomputed_ds, open("/tmp/test1/ds.pkl", "wb"))

In [ ]:
import pickle
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

n = 50
tmpdirname = "/tmp/test1"
fn = tmpdirname + f"/perfect_circle_n{n}_noise0p0"

v5inf = v5inferencedataset(
    yaml_fn=fn + ".yaml",
    nthetas=65,
    gpu=False,
    n_parallel=8,
    paired=True,
    model_config_fn="",
    vehicle_type="wallarray",
    skip_segmentation=True,
    skip_detrend=True,
)
precomputed_ds = pickle.load(open("/tmp/test1/ds.pkl", "rb"))
for idx in range(20):
    d = precomputed_ds[idx]
    for ridx in range(2):
        print(idx, ridx)
        v5inf.write_to_idx(idx, ridx, data_single_radio_to_raw(d[ridx]))

In [ ]:
ds = v5spfdataset(  # make sure everything gets segmented here
    fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    skip_fields=set(["signal_matrix"]),
    segment_if_not_exist=True,
    v4=True,
)

In [ ]:
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

import pickle
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

# create_fake_dataset(filename=fn, yaml_config_str=fake_yaml_v4, n=50, noise=0.01)

n = 50
tmpdirname = "/tmp/test1"
fn = tmpdirname + f"/perfect_circle_n{n}_noise0p0"

ds = v5spfdataset(  # make sure everything gets segmented here
    fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    # skip_fields=set(["signal_matrix"]),
    segment_if_not_exist=True,
    v4=True,
)

v5inf = v5inferencedataset(
    yaml_fn=fn + ".yaml",
    nthetas=65,
    gpu=False,
    n_parallel=8,
    paired=True,
    model_config_fn="",
    skip_fields=[],
    vehicle_type="wallarray",
    skip_segmentation=False,
    skip_detrend=False,
)
n = 3

for idx in range(n):
    d = ds[idx]
    for ridx in range(2):
        print(idx, ridx)
        v5inf.write_to_idx(idx, ridx, data_single_radio_to_raw(d[ridx]))
        # compare_two_entries(ds[0][1], v5inf[0][1])

In [ ]:
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

import pickle
from spf.dataset.spf_dataset import v5inferencedataset, data_single_radio_to_raw
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v4
from spf.dataset.spf_dataset import v5spfdataset

# create_fake_dataset(filename=fn, yaml_config_str=fake_yaml_v4, n=50, noise=0.01)
import logging
from spf.dataset.spf_dataset import v5_collate_keys_fast, global_config_to_keys_used
import torch

# try doing neural net inference online
# get paired checkpoint results
from spf.model_training_and_inference.models.single_point_networks_inference import (
    load_model_and_config_from_config_fn_and_checkpoint,
)

paired_checkpoint_fn = "/home/mouse9911/gits/spf/checkpoints/march16/paired_wd0p02_gains_vehicle_0p2dropout_noroverbounceREAL_lowdrop_x2/best.pth"
paired_config_fn = "/home/mouse9911/gits/spf/checkpoints/march16/paired_wd0p02_gains_vehicle_0p2dropout_noroverbounceREAL_lowdrop_x2/config.yml"
empirical_dist_fn = "/home/mouse9911/gits/spf/checkpoints/apr5/single_wd0p02_gains_vehicle_0p2dropout_noroverbounceREAL_lowdrop_x2_updateddata_lowerdrop_rerun_3p6/empirical_dist.pkl"
inference_cache = "/tmp/infcache"
n = 50
tmpdirname = "/tmp/test1"
fn = tmpdirname + f"/perfect_circle_n{n}_noise0p0"

ds = v5spfdataset(  # make sure everything gets segmented here
    fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    empirical_data_fn=empirical_dist_fn,
    # skip_fields=set(["signal_matrix"]),
    segment_if_not_exist=True,
    v4=True,
)

v5inf = v5inferencedataset(
    yaml_fn=fn + ".yaml",
    nthetas=65,
    gpu=False,
    n_parallel=8,
    paired=True,
    model_config_fn="",
    empirical_data_fn=empirical_dist_fn,
    skip_fields=["signal_matrix"],
    vehicle_type="wallarray",
    skip_segmentation=False,
    skip_detrend=False,
)
n = 3

for idx in range(n):
    d = ds[idx]
    for ridx in range(2):
        print(idx, ridx)
        # v5inf.write_to_idx(idx, ridx, data_single_radio_to_raw(d[ridx]))
        v5inf.write_to_idx(idx, ridx, d[ridx])
        # compare_two_entries(ds[0][1], v5inf[0][1])

In [ ]:
from spf.dataset.spf_nn_dataset_wrapper import v5spfdataset_nn_wrapper


nn_ds = v5spfdataset_nn_wrapper(
    v5inf,
    paired_config_fn,
    paired_checkpoint_fn,
    inference_cache,
    device="cpu",
)

In [ ]:
ds = v5spfdataset(  # make sure everything gets segmented here
    fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    empirical_data_fn=empirical_dist_fn,
    # skip_fields=set(["signal_matrix"]),
    segment_if_not_exist=True,
    v4=True,
)

In [ ]:
from spf.model_training_and_inference.models.single_point_networks_inference import (
    get_nn_inference_on_ds_and_cache,
)
import os
import torch

os.makedirs(inference_cache, exist_ok=True)

cached_model_inference = get_nn_inference_on_ds_and_cache(
    ds_fn=ds.zarr_fn,
    config_fn=paired_config_fn,
    checkpoint_fn=paired_checkpoint_fn,
    device="cpu",
    inference_cache=inference_cache,
    batch_size=64,
    workers=0,
    precompute_cache=ds.precompute_cache,
    crash_if_not_cached=False,
    segmentation_version=ds.segmentation_version,
    v4=True,
)["paired"]
type(cached_model_inference)

from spf.dataset.spf_nn_dataset_wrapper import v5spfdataset_nn_wrapper


nn_ds = v5spfdataset_nn_wrapper(
    ds,
    paired_config_fn,
    paired_checkpoint_fn,
    inference_cache,
    device="cpu",
)

In [ ]:
nn_ds[0][0]["paired"].shape

In [ ]:
get_nn_inference_on_ds_and_cache(
    ds_fn=ds.zarr_fn,
    config_fn=paired_config_fn,
    checkpoint_fn=paired_checkpoint_fn,
    device="cpu",
    inference_cache=inference_cache,
    batch_size=64,
    workers=0,
    precompute_cache=ds.precompute_cache,
    crash_if_not_cached=False,
    segmentation_version=ds.segmentation_version,
    v4=True,
)["paired"][0]

In [ ]:
from spf.dataset.spf_dataset import get_empirical_dist

get_empirical_dist(v5inf, 0).T.unsqueeze(0)

In [ ]:
v5inf[0][0]["mean_phase_segmentation"]

In [ ]:
from spf.filters.particle_dualradio_filter import plot_single_theta_dual_radio
from spf.filters.run_filters_on_data import run_PF_single_theta_dual_radio


args = {
    "ds": ds,
    "N": 1024 * 4,
    "theta_err": 0.01,
    "theta_dot_err": 0.01,
}
result = run_PF_single_theta_dual_radio(**args)
assert result[0]["metrics"]["mse_craft_theta"] < 0.15
plot_single_theta_dual_radio(ds)